In [1]:
from keras.preprocessing import sequence
from keras.models import Sequential
from keras.layers import Dense, Dropout, Activation
import pandas as pd
import nltk
from sklearn.model_selection import train_test_split
from sklearn import metrics
from sklearn.svm import LinearSVC, SVC
from sklearn.naive_bayes import MultinomialNB
from sklearn.feature_extraction.text import TfidfVectorizer
import string
from nltk import pos_tag
from nltk.corpus import stopwords
from nltk.corpus import wordnet
import re
import gensim
from gensim.models import KeyedVectors
import numpy as np
from collections import Counter
from numpy import dot
from numpy.linalg import norm
from gensim.models import Word2Vec
from gensim.models.phrases import Phrases
from nltk import sent_tokenize
from nltk.tokenize.treebank import TreebankWordDetokenizer
import gensim.downloader as api
from sklearn.model_selection import GridSearchCV, StratifiedKFold
from sklearn.metrics import accuracy_score, f1_score, classification_report
from sklearn.metrics import accuracy_score, precision_score, recall_score, make_scorer
from preprocessing import *

In [2]:
glove_twitter_25 = []

In [3]:
# print("loading pre trained embeddings, this can take some minutes...")
# glove_twitter_25 = KeyedVectors.load_word2vec_format('glove-twitter-25.txt', binary=False)
# print("loading complete.")

In [4]:
dataset = pd.read_csv("amazon_alexa.tsv", sep = "\t", encoding = "utf-8")
print(dataset.shape)
dataset.dropna(inplace = True)
print(dataset.shape)
dataset.drop(dataset[dataset.rating == 3].index, inplace=True)
print(dataset.shape)
dataset.drop_duplicates(subset = "verified_reviews", inplace = True)
print(dataset.shape)

(3150, 5)
(3150, 5)
(2998, 5)
(2196, 5)


In [5]:
X = np.array(dataset["verified_reviews"].values).reshape(-1, 1)
y = list(dataset["feedback"].values)

In [6]:
from imblearn.under_sampling import RandomUnderSampler
from collections import Counter

undersampler = RandomUnderSampler(sampling_strategy=0.5, random_state = 0)

X, y = undersampler.fit_resample(X, y)


print('Resampled dataset shape %s' % Counter(y))

Resampled dataset shape Counter({1: 412, 0: 206})


In [7]:
X_temp = []

for rev in X:
  X_temp.append(rev[0])

In [8]:
new_text, new_sent_tok = tokenize_list_of_text(X_temp, custom_stopwords, True, 2)

total number of types extracted is: 2693


In [9]:
def frequency_cleaning(new_sent_tok, n):
    
    tot_tokens = []

    for sent in new_sent_tok:
        for tok in sent:
            tot_tokens.append(tok)

    freqs = nltk.FreqDist(tot_tokens)
    cleaned_reviews = []

    for sent in new_sent_tok:
        clean_sent = []
        for tok in sent:
            if freqs[tok] > n:
                clean_sent.append(tok)
        cleaned_reviews.append(clean_sent)

    return cleaned_reviews

In [10]:
cleaned_reviews = frequency_cleaning(new_sent_tok, 2)

In [11]:
bigrams = Phrases(cleaned_reviews, scoring="npmi", threshold=0.60) #estrae le collocazioni tramite PMI
bigrams[cleaned_reviews][0]

['fact_NN',
 'answer_NNS',
 'not_see_VB',
 'real_JJ',
 'household_NN',
 'though_IN',
 'good_JJ',
 'day_NN']

In [12]:
X_train, X_test, Y_train, Y_test = train_test_split(bigrams[cleaned_reviews], y, test_size=0.20, random_state=10)

In [13]:
tfidf = TfidfVectorizer(tokenizer=lambda i:i, lowercase=False, min_df = 0)
tfidf_model = tfidf.fit_transform(X_train)
X_test_tf = tfidf.transform(X_test)
review = 0
for score, feature in zip(tfidf_model.toarray()[review], tfidf.get_feature_names_out()):
    if score > 0.0:
        print(feature, score)

box_NN 0.2693722673908957
even_RB_though_IN 0.2693722673908957
far_RB 0.20215600784785842
happy_JJ 0.4753244285386046
like_IN 0.15763986748133363
look_VBZ 0.25649877931002296
new_JJ 0.2041650742860883
original_JJ 0.251101726097465
perfectly_RB 0.2772685786094323
purchase_NN 0.25649877931002296
refurbish_VBN 0.251101726097465
two_CD 0.23386606096945184
week_NNS 0.2866079325189088
work_VBG 0.22392049137031347


In [14]:
w2v_model = Word2Vec(X_train, vector_size=100, window = 10, min_count = 0, sg=1, hs = 1, seed = 5, epochs=100)

In [15]:
w2v_model.wv.most_similar("love_VB", topn = 10)

[('great_JJ', 0.4103817641735077),
 ('command_NNS', 0.3745611906051636),
 ('love_VBP', 0.3717384934425354),
 ('good_JJ', 0.34893396496772766),
 ('not_problem_NN', 0.3488442301750183),
 ('game_NNS', 0.3429841697216034),
 ('play_VBG', 0.33462899923324585),
 ('favorite_JJ', 0.3295687139034271),
 ('song_NNS', 0.32041609287261963),
 ('sometimes_RB', 0.315849244594574)]

In [16]:
w2v_model.wv.most_similar("love_VB", topn = 2000)[-10:]

[('research_NN', -0.09369165450334549),
 ('alarm_NN_clock_NN', -0.09507547318935394),
 ('stop_VBD', -0.09683091938495636),
 ('hop_VBG', -0.10076894611120224),
 ('already_RB', -0.10347534716129303),
 ('enough_RB', -0.10750862210988998),
 ('device_NN', -0.10778271406888962),
 ('give_VBN', -0.1325184851884842),
 ('price_NN', -0.14415259659290314),
 ('differ_VB', -0.14933691918849945)]

In [17]:
def review_vectors(tokens, size = 300, weights = [], pretrained = False, pretrained_embeddings = None):
    
    """Genera un vettore per ogni recensione: questo vettore
    è calcolato come la media ponderata (t * w: token vettore * peso tfidf) dei vettori dei token nella recensione.
    Il vettore risultante è normalizzato alla fine."""

    vec = np.zeros(size).reshape((1, size))
    count = 0

    for word, weight in zip(tokens, weights):
        try:
            if pretrained:
                vec += pretrained_embeddings[word] 
                count +=1
            else:
                vec += w2v_model.wv[word]
                count +=1
        except KeyError:
            # print("non trovo", word)
            continue

    if count!= 0:
        vec = vec / norm(vec)
        
    return vec

In [18]:
w2v_model.vector_size

100

In [19]:
w2v_X_train = np.zeros((len(X_train), w2v_model.vector_size))
for i in range(len(X_train)):
    w2v_X_train[i,:] = review_vectors(tfidf.inverse_transform(tfidf_model[i, :])[0], w2v_model.vector_size, tfidf_model[i,:].data, False, glove_twitter_25)
w2v_X_train.shape

(494, 100)

In [20]:
w2v_X_test = np.zeros((len(X_test), w2v_model.vector_size))
for i in range(len(X_test)):
    w2v_X_test[i,:] = review_vectors(tfidf.inverse_transform(X_test_tf[i, :])[0], w2v_model.vector_size, X_test_tf[i,:].data, False, glove_twitter_25)
w2v_X_test.shape

(124, 100)

In [21]:
from imblearn.over_sampling import SMOTE
from imblearn.under_sampling import RandomUnderSampler, TomekLinks

# # Create the SMOTE object
# sm = SMOTE(sampling_strategy=1, k_neighbors=5, random_state=10)

# # Apply SMOTE to the training data
# X_train_resampled, y_train_resampled = sm.fit_resample(w2v_X_train, Y_train)

# print('Resampled dataset shape %s' % Counter(y_train_resampled))

undersampler_nn = TomekLinks(sampling_strategy="all") # use tomeLinks to remove ambiguous data through nearest neighbours


# Fit and transform the X and y data
# X_train_resampled, y_train_resampled = undersampler_nn.fit_resample(w2v_X_train, Y_train)

X_train_resampled, y_train_resampled = w2v_X_train, Y_train


print('Resampled dataset shape %s' % Counter(y_train_resampled))

Resampled dataset shape Counter({1: 329, 0: 165})


In [22]:
from keras.layers import Flatten

In [23]:
from keras.utils import to_categorical
Y_train_hot = to_categorical(y_train_resampled)

In [24]:
Y_test_hot = to_categorical(Y_test)

In [31]:
from keras.callbacks import EarlyStopping

# Define the early stopping criteria
early_stopping = EarlyStopping(monitor='binary_accuracy', patience=25)

# Fit the model with early stopping
model = Sequential()
model.add(Dense(300, activation='relu', input_dim=w2v_model.vector_size))
model.add(Dropout(0.2))
model.add(Dense(300, activation='relu'))
model.add(Dropout(0.2))
model.add(Dense(100, activation='relu'))
model.add(Dropout(0.2))
model.add(Dense(2, activation='sigmoid'))

# Compile your model
model.compile(loss='binary_crossentropy', optimizer='adam', metrics=['binary_accuracy'])

In [32]:
model.fit(X_train_resampled, Y_train_hot, epochs=100, batch_size=12, verbose=2, validation_split=0.1, callbacks = [early_stopping])

Epoch 1/100
37/37 - 1s - loss: 0.6095 - binary_accuracy: 0.6588 - val_loss: 0.5473 - val_binary_accuracy: 0.6900 - 1s/epoch - 28ms/step
Epoch 2/100
37/37 - 0s - loss: 0.3973 - binary_accuracy: 0.8198 - val_loss: 0.4916 - val_binary_accuracy: 0.8000 - 112ms/epoch - 3ms/step
Epoch 3/100
37/37 - 0s - loss: 0.2791 - binary_accuracy: 0.8998 - val_loss: 0.4976 - val_binary_accuracy: 0.8200 - 114ms/epoch - 3ms/step
Epoch 4/100
37/37 - 0s - loss: 0.2260 - binary_accuracy: 0.9110 - val_loss: 0.4700 - val_binary_accuracy: 0.8000 - 136ms/epoch - 4ms/step
Epoch 5/100
37/37 - 0s - loss: 0.1908 - binary_accuracy: 0.9245 - val_loss: 0.4224 - val_binary_accuracy: 0.7800 - 126ms/epoch - 3ms/step
Epoch 6/100
37/37 - 0s - loss: 0.1390 - binary_accuracy: 0.9516 - val_loss: 0.5604 - val_binary_accuracy: 0.8200 - 112ms/epoch - 3ms/step
Epoch 7/100
37/37 - 0s - loss: 0.1345 - binary_accuracy: 0.9471 - val_loss: 0.4669 - val_binary_accuracy: 0.8000 - 134ms/epoch - 4ms/step
Epoch 8/100
37/37 - 0s - loss: 0.083

In [33]:
model.summary()

Model: "sequential_1"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 dense_4 (Dense)             (None, 300)               30300     
                                                                 
 dropout_3 (Dropout)         (None, 300)               0         
                                                                 
 dense_5 (Dense)             (None, 300)               90300     
                                                                 
 dropout_4 (Dropout)         (None, 300)               0         
                                                                 
 dense_6 (Dense)             (None, 100)               30100     
                                                                 
 dropout_5 (Dropout)         (None, 100)               0         
                                                                 
 dense_7 (Dense)             (None, 2)                

In [34]:
predictions = model.predict(w2v_X_test) 

4/4 [==============================] - 0s 2ms/step


In [35]:
predictions = np.round(predictions)

In [36]:
from sklearn.metrics import classification_report
print(classification_report(Y_test_hot, predictions))

              precision    recall  f1-score   support

           0       0.69      0.71      0.70        41
           1       0.85      0.84      0.85        83

   micro avg       0.80      0.80      0.80       124
   macro avg       0.77      0.78      0.77       124
weighted avg       0.80      0.80      0.80       124
 samples avg       0.80      0.80      0.80       124

